# Neural Relation Extraction with PyTorch

Relation extraction is an information extraction task that aims to detect semantic relationships between pairs of entities in a given text. The entities can belong to different types (person, location, date...) and relationships can differ in nature (contains, belongs...). This task is essential for building knowledge graphs which are a graph structure representing the relationships between large numbers of entities. These graphs have wide applications such as search engines, recommendation systems and healthcare.

In this post, we will demonstrate Neural Relation Extraction (NRE) using an LSTM-based neural network for a set of given sentences. We will use the data from the [SemEval-2010 Task 8](https://docs.google.com/document/d/1QO_CnmvNRnYwNWu1-QCAeR5ToQYkXUqFeAJbdEhsq7w/preview). Each sample in this dataset is a sentence where the two entities in question are tagged. The semantic relationship with each sentence is provided. There are nine different kinds of relationships in this dataset in addition to the "Other" class which makes for ten total classes:
1. Cause-Effect  
2. Instrument-Agency
3. Product-Producer
4. Content-Container
5. Entity-Origin
6. Entity-Destination
7. Component-Whole
8. Member-Collection
9. Message-Topic

An example from this dataset is shown below:

*The \<e1>factory\</e1>'s products have included flower pots, Finnish rooster-whistles, pans, \<e2>trays\</e2> tea pots, ash trays and air moisturisers.*

*Product-Producer(e2,e1)*

In the example above, the two entities we are concerned about are marked with \<e1> and \<e2> tags and the corresponding relationship is provided.

There are many approaches for solving the relation extraction problem in NLP. In this post, we will test two bidirectional LSTM approaches. The first is a simple BiLSTM that takes as input the sentences as sequences of words. We learn word embeddings to use as input features for the LSTM. For the second approach, we build two word position vectors for each sentence indicating the relative distance of each word with respect to each entity. We also learn high dimensional embeddings for these relative positions and concatenate them to the word embeddings. Including word position vectors is a technique that has been shown to improve performance since otherwise the model would not know which are the entities in concern. This approach has been adopted in many research works such as [here](https://aclanthology.org/P16-1123/).

## Data Preprocessing
Before we dive into our NRE model, we must first load and preprocess the data to prepare it for our task.

The below cell downloads the dataset from the link provided by SemEval and exracts it.

In [ ]:
!gdown --fuzzy https://drive.google.com/file/d/0B_jQiLugGTAkMDQ5ZjZiMTUtMzQ1Yy00YWNmLWJlZDYtOWY1ZDMwY2U4YjFk/view?sort=name&layout=list&num=50&resourcekey=0-k0OTSIGrF9UAcrTFfInlrw
!unzip -q SemEval2010_task8_all_data.zip
!rm SemEval2010_task8_all_data.zip
!mv SemEval2010_task8_all_data/SemEval2010_task8_training/TRAIN_FILE.TXT TRAIN_FILE.TXT
!mv SemEval2010_task8_all_data/SemEval2010_task8_testing_keys/TEST_FILE_FULL.TXT TEST_FILE_FULL.TXT
!rm -r SemEval2010_task8_all_data

If we look at the text structure in this file, we notice that it is not structured in a uniform manner. Each sample begins with a sentence ID followed by a tabspace and the sentence. The relationship is then provided on a new line, followed by comment regarding the relationship annotation. Samples are separated by two new lines.

In [ ]:
with open('TRAIN_FILE.TXT', 'r') as f:
    print(f.read())

We would like to load the dataset into a Pandas DataFrame which will make further manipulations faster and easier. To be able to read the text into a DataFrame, we must change the text structure into a format that Pandas can parse. To do this, we use regular expressions replace the single new line characters separating the sentence from the relationship and the relationship from the comment with a tabspace. As a result, the dataset will become a tab-separated-values (tsv) file which can be easily read by Pandas. We do this for both the train and test files.

In [ ]:
import io
import re
import pandas as pd

with open('TRAIN_FILE.TXT', 'r') as f:
    text = re.sub(r'(?<!\n)\n(?!\n)', '\t', f.read()) #format the text file to tsv
train = pd.read_csv(io.StringIO(text), sep='\t', header=None, names=['id','sent','rel','comment'], index_col=0).drop(columns=['comment'])

#do the same for test
with open('TEST_FILE_FULL.TXT', 'r') as f:
    text = re.sub(r'(?<!\n)\n(?!\n)', '\t', f.read())
test = pd.read_csv(io.StringIO(text), sep='\t', header=None, names=['id','sent','rel','comment'], index_col=0).drop(columns=['comment'])

train.head()

When looking at the class distribution, we find that some relationships are present twice due to the difference in the relationship direction (e1 to e2 or e2 to e1). To simplify the task, we only consider the detection of the nature of the relationship and not its direction. Therefore, we remove the direction information between parenthesis.

In [ ]:
train['rel'].value_counts()

In [ ]:
#unify relationships by removing relationship direction between parenthesis
train['rel'] = train['rel'].apply(lambda x: re.sub(r'\([^()]*\)', '', x))
test['rel'] = test['rel'].apply(lambda x: re.sub(r'\([^()]*\)', '', x))

train['rel'].value_counts()

The next step is to clean up the text from tokens that do not contribute to the learning such as punctuation and symbols. However, we exclude angle brackets (<>) since these will allow us to extract entity positions.

We also convert all characters to lowercase because otherwise words with different capitalization will be considered as different words.

It is worth mentioning that one text cleanup step that was not done here is stopword removal. Stopwords such as prepositions are usually removed since they are used in all text regardless of its meaning. However, in the case of relation extraction, we decided to keep them since they can indicate the nature of a relationship.

In the example **"Jack is in the cafeteria."**, the preposition **"in"** indicates a location relationship.

In [ ]:
#remove punctuation except subject and object tags
punc = """!"#$%&'()*+,-./:;=?@[\]^_`{|}~"""
train['sent'] = train['sent'].apply(lambda x: x.translate(str.maketrans('', '', punc)))
test['sent'] = test['sent'].apply(lambda x: x.translate(str.maketrans('', '', punc)))

#lowercase
train['sent'] = train['sent'].apply(lambda x: x.lower())
test['sent'] = test['sent'].apply(lambda x: x.lower())

In the following cell, we use the entity tags to add two columns to the dataset: position of entity 1 (e1) and position of entity 2 (e2). This could not have been done before cleaning the text since some punctuation and symbols might get counted as tokens and shift the word positioning. After extracting the entity positions for each sentence, we remove the entity tags from the text.

In [ ]:
#extract subject and object word positions
def e1_pos(sent):
    for word in sent.split(' '):
        if re.match(r'.*<e1>.*', word):
            return sent.split(' ').index(word)
def e2_pos(sent):
    for word in sent.split(' '):
        if re.match(r'.*<e2>.*', word):
            return sent.split(' ').index(word)
train['e1'] = train['sent'].apply(e1_pos)
train['e2'] = train['sent'].apply(e2_pos)
test['e1'] = test['sent'].apply(e1_pos)
test['e2'] = test['sent'].apply(e2_pos)

#remove subject and object tags
train['sent'] = train['sent'].apply(lambda x: re.sub(r'<.+?>', '', x))
test['sent'] = test['sent'].apply(lambda x: re.sub(r'<.+?>', '', x))

We are now ready to prepare the text for machine learning. Our goal here is to map each word in the vocabulary of the entire dataset to a unique integer identity. Therefore, each sentence will be represented by a sequence of integers. The overall dataset will be represented by a matrix of dimension (M,N) where M is the number of samples and N is the sequence length (number of words in the sentence). Each scalar in the N-sized vector is an integer representing a word. Since not all sentences have the same length, we pad all shorter sentences by zeros to reach the length of the maximum sentence in the dataset.

For the procedure described above, we perform the following steps:
- Compile a list of the full dataset vocabulary.
- Exclude the least common words from the vocab. These will be represented by the special token ID 1.
- Create a word to index mapping to assign each word in the vocabulary a unique ID. Note that we start the mapping from 2 since the 0 and 1 indexes are reserved for padding and unknown tokens respectively.
- To do this transformation, we define a function that takes a DataFrame and outputs a matrix where all sentences are padded to the same length and each word is replaced by its corresponding ID (we compute the maximum sentence length across both train and test sets since it's necessary to build the matrix).

In [ ]:
from collections import Counter

#build full list of tokens
tokens = []
for row in train.itertuples():
    for word in row.sent.split(' '):
        tokens.append(word)
for row in test.itertuples():
    for word in row.sent.split(' '):
        tokens.append(word)

#remove uncommon tokens
counts = dict(Counter(tokens))
vocab = [word for word in counts if counts[word] > 10]

#word integer encoding
#0 is for padding
#1 is for unknown
word2id = {word: id+2 for id, word in enumerate(sorted(vocab))}
id2word = {id+2: word for id, word in enumerate(sorted(vocab))}


#max sentence length
max_len_train = max(len(row.sent.split(' ')) for row in train.itertuples())
max_len_test = max(len(row.sent.split(' ')) for row in test.itertuples())
max_len = max(max_len_train, max_len_test)

#word index matrix
def index_words(data):
    X = torch.zeros((data.shape[0], max_len))
    for row_index, row in enumerate(data.itertuples()):
        for word_pos, word in enumerate(row.sent.split(' ')):
            try:
                X[row_index, word_pos] = word2id[word]
            except KeyError:
                X[row_index, word_pos] = 1
    return X.int()

We have now fully preprocessed and prepared the sentences for our model. Now we need to prepare our other inputs: the relative word positions with respect to both entities. This can be easily done using the entity word positions we extract previously.

In our position vectors, the position of the entity is given a value of 1 and all other words going in the forward and backward directions are incremented by 1.

Let's consider the below example from the dataset:

*A large \<e1>marble\</e1> was dropped into the \<e2>bowl\</e2>.*

*Entity-Destination(e1,e2)*

The word position vectors for this sentence would be the following:

|  | a | large | marble | was | dropped | into | the | bowl |
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
| marble | 3 | 2 | 1 | 2 | 3 | 4 | 5 | 6 |
| bowl | 8 | 7 | 6 | 5 | 4 | 3 | 2 | 1 |

One thing to note is that we could have represented distances in the backwards direction as negative numbers. However, the Embedding layer in Pytorch cannot index using negative numbers.

As with sentences, word position vectors are padded to the maximum length. We create the below function to generate the word position vectors.

In [ ]:
import torch

#distance for each word from subject and object words
def word_distances(data):
    X_pos1 = torch.zeros((data.shape[0], max_len))
    X_pos2 = torch.zeros((data.shape[0], max_len))
    for row_index, row in enumerate(data.itertuples()):
        X_pos1[row_index, :len(row.sent.split(' '))] = torch.abs(torch.arange(-row.e1,len(row.sent.split(' '))-row.e1)) + 1
        X_pos2[row_index, :len(row.sent.split(' '))] = torch.abs(torch.arange(-row.e2,len(row.sent.split(' '))-row.e2)) + 1
    return X_pos1.int(), X_pos2.int()

The final step in our data preparation pipeline is to split the training set into a training and validation set. The latter will be used for model selection while the predefined test set will be used to assess the performance of the final model. We shuffle the dataset before splitting to eliminate any bias.

Afterwards, we represent the relationship labels in one-hot encoding format.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

#shuffle training data
train = train.sample(frac=1, random_state=41)

#split into train and val (80/20%)
train_idx, val_idx = train_test_split(range(train.shape[0]), test_size=0.2, random_state=41, stratify=train['rel'].values)
val = train.iloc[val_idx,:]
train = train.iloc[train_idx,:]

#extract and one-hot labels
ohe = OneHotEncoder(sparse=False)
y_train = torch.from_numpy(ohe.fit_transform(train['rel'].values.reshape(-1, 1)))
y_val = torch.from_numpy(ohe.transform(val['rel'].values.reshape(-1, 1)))
y_test = torch.from_numpy(ohe.transform(test['rel'].values.reshape(-1, 1)))

## First Model

As described in the beginning, the first approach is an LSTM which takes as input sentences as sequences of words and outputs the relationship classification after having consumed the whole sequence.

Each word is represented by an embedding vector. Word embeddings are learned by an embedding layer during the training.

The shape of the embedding matrix should be (number of elements to embed, embedding vector dimension). The embedding vector dimension is a hyperparameter that can be tuned. In this case, the number of elements to embed is set to the vocab size (including 0 and 1 tokens) since we need a unique embedding vector for each token in the vocabulary. We also set the argument *padding_idx* to 0 to indicate that tokens of ID 0 are meant for padding and can be ignored when learning the embeddings.

Before passing the output of the embedding layer to the BiLSTM, we pass it through the "pack_padded_sequence" function. This function packs and batches the data in a way to ignore the zero indexes corresponding to the padding tokens. This results in a considerable computation reduction. To do this, we need to provide the actual length for each sequence (prior to padding). Based on this information, the function will basically sort the samples and divide them into batches based on actual length. For a full explanation of the workings of this function refer to the tutorial [here](https://github.com/sgrvinod/a-PyTorch-Tutorial-to-Sequence-Labeling).

The next layer in our model is a bidirectional LSTM which consumes the embedding vector. The BiLSTM is in fact training two LSTM models with one consuming the sequence in reverse order. The output at the last timestep is a concatenation of the outputs for both LSTMs. The LSTM typically outputs three things for every sequence:
- The output at each timestep.
- The last hidden state.
- The last cell state.

In the case of BiLSTM, the last hidden and cell states include states from both LSTMs (at the first and last word in the sequence). For our task, we only need the last hidden state to be fed into a dense softmax classification layer. The shape of the hidden state output is (2, batch_size, hidden_size) while a dense layer expects an input of shape (batch_size, input_size). The hidden state has an additional dimension of size 2 due to the presence of two LSTMs for our bidirectional model. We therefore reshape the hidden state by collapsing the additional dimension into the last dimension. As a result, the new hidden state shape would be (batch_size, hidden_size × 2).

Finally, the classification layer produces a softmax output using the last hidden state. The diagram below shows the shapes of the word embeddings and the architecture of the model.

![picture](https://drive.google.com/uc?id=1pR10HYQiVnwWr1i5urh7cgZJ1GmglNeK)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

#define model
class NRE1(nn.Module):
    def __init__(self, max_sent_len, vocab_size, hunits, embed_dim, n_classes, dropout_rate):
        super().__init__()
        self.dropout_rate = dropout_rate
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(embed_dim, hunits, batch_first=True, bidirectional=True)
        self.out = nn.Linear(hunits*2, n_classes)
    
    def forward(self, X, lengths):
        X = self.embedding(X)
        X = F.dropout(X, p=self.dropout_rate, training=self.training)
        X = nn.utils.rnn.pack_padded_sequence(X, lengths, batch_first=True, enforce_sorted=False).to(device)
        out, (hn, cn) = self.lstm(X)
        X = torch.cat((hn[0],hn[1]), dim=1)
        X = F.dropout(X, p=self.dropout_rate, training=self.training)
        X = self.out(X)
        return F.log_softmax(X, dim=1)

## Training

We are now finally ready to train our model. The code below allows to train in batches to accomodate for memory limitations.

As for typical classification problems, we use the categorical crossentropy loss function.  For our evaluation metric, we use the macro-averaged F1 score as indicated by the SemEval competition rules. The F1 score is a reasonable metric here since the classes are not balanced and using the accuracy would be misleading.

For every epoch, we record the training loss and F1 score. We also evaluate the model on the validation set after every epoch and also record the loss and F1 score. After the training is finished we plot the training and validation metrics to assess model bias and variance.

In [ ]:
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(41)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

#hyperparameters
num_epochs = 12
batch_size = 64
hunits = 256
embedding_dim = 200
dropout_rate = 0.5
lr = 0.003

#model
model = NRE1(max_len, len(vocab)+2, hunits, embedding_dim, y_train.shape[1], dropout_rate)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

#evaluation function
def evaluate1(data, targets, return_out=False):
    X = index_words(data).to(device)
    lengths = [len(row.sent.split(' ')) for row in data.itertuples()]
    with torch.no_grad():
        model.eval()
        out = model(X, lengths).cpu()
        loss = F.nll_loss(input=out, target=targets.argmax(dim=1))
        f1 = f1_score(targets.argmax(dim=1), out.argmax(dim=1), average='macro')
    if return_out:
        return loss, f1, out
    else:
        return loss, f1

#training loop
train_losses = []
train_f1s = []
val_losses = []
val_f1s = []
for epoch in range(num_epochs):
    #training
    loss_sum = 0
    train_preds = []
    train_targets = []
    model.train()
    for i in range(0, train.shape[0], batch_size):
        X_train_batch = index_words(train.iloc[i:i+batch_size]).to(device)
        lengths = [len(row.sent.split(' ')) for row in train.iloc[i:i+batch_size].itertuples()]
        y_train_batch = y_train[i:i+batch_size].to(device)
        train_targets.append(y_train_batch.cpu())
        optimizer.zero_grad()
        out = model(X_train_batch, lengths)
        train_preds.append(out.detach().cpu())
        loss = F.nll_loss(input=out, target=y_train_batch.argmax(dim=1))
        loss.backward()
        optimizer.step()
        loss_sum += loss.detach().item()
    train_targets = torch.cat(train_targets)
    train_preds = torch.cat(train_preds)
    f1_train = f1_score(train_targets.argmax(dim=1), train_preds.argmax(dim=1), average='macro')
    train_f1s.append(f1_train)
    train_losses.append(loss_sum/batch_size)
    #validation
    loss_val, f1_val = evaluate1(val, y_val)
    val_losses.append(loss_val)
    val_f1s.append(f1_val)
    #epoch end
    print(f'********Epoch {epoch+1}********')
    print(f'Train loss: {loss_sum/batch_size:.2f}, Train F1: {f1_train:.2f};    Val loss {loss_val:.2f}, Val F1 {f1_val:.2f}\n')

#training end
plt.figure()
plt.plot(train_losses)
plt.plot(val_losses)
plt.legend(['train', 'val'])
plt.xlabel('epochs')
plt.ylabel('loss')
plt.show()

plt.figure()
plt.plot(train_f1s)
plt.plot(val_f1s)
plt.legend(['train', 'val'])
plt.xlabel('epochs')
plt.ylabel('F1')
plt.show()

## Testing

After having selected our final model configuration above based on its performance on the validation set, we evaluate our model on the testing set which would give us a final assessment of our model. This is the same set that was used to rank submissions during the SemEval competition.

We additionally plot the confusion matrix to further understand the strengths and shortcomings of our model.

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

loss_test, f1_test, out = evaluate1(test, y_test, return_out=True)
print(f'Test loss: {loss_test:.2f}, Test F1: {f1_test:.2f}\n')
ConfusionMatrixDisplay.from_predictions(
    y_test.argmax(dim=1),
    out.argmax(dim=1),
    display_labels=ohe.categories_[0],
    xticks_rotation='vertical',
    cmap='Blues'
);

## Second Model

In the second approach, we introduce word position embeddings which indicate to the model which are the entity word in question.

Each word is represented by a vector combining its corresponding word embedding, its position embeddings with respect to the first entity and its position embedding with respect to the second entity. Therefore, as our model input we define three embedding layers.

For the word position embedding layer, we set the number of elements to the maximum sentence length. This is considering the extreme case when the we have the longest possible sentence and the entity is at the beginning/end of the sentence. Therefore the last/first word would have a distance equal to the maximum sentence length and this is the largest relative position we will have to embed.

The shape of the output embedding is (batch_size, max_sequence_length, embedding_dimension). Therefore, we concatenate the outputs of the three embedding layers along the third dimension since we are concatenating at the word level and not the sentence level.

We do not use "pack_padded_sequences" in this case since the way it is currently implemented would result in discarding not only the padding tokens but the word position embeddings as well. The code can be further customized to address this need, but to keep things simple and because our dataset is not very big we use the embeddings without packing.

The diagram below shows the architecture of this model highlighting the format and handling of the introduced word position embeddings.

![picture](https://drive.google.com/uc?id=1wXgum1CX8ohY9Je47gVxq9jPeAdsFieg)

In [ ]:
class NRE2(nn.Module):
    def __init__(self, max_sent_len, vocab_size, hunits, word_embed_dim, pos_embed_dim, dropout_rate, n_classes):
        super().__init__()
        self.dropout_rate = dropout_rate
        self.embedding = nn.Embedding(vocab_size, word_embed_dim, padding_idx=0)
        self.pos1 = nn.Embedding(max_sent_len, pos_embed_dim, padding_idx=0)
        self.pos2 = nn.Embedding(max_sent_len, pos_embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(word_embed_dim+2*pos_embed_dim, hunits, batch_first=True, bidirectional=True)
        self.out = nn.Linear(hunits*2, n_classes)
    
    def forward(self, X, pos1, pos2):
        pos1 = self.pos1(pos1)
        pos2 = self.pos2(pos2)
        X = self.embedding(X)
        input = torch.cat((X,pos1,pos2), dim=2)
        input = F.dropout(input, p=self.dropout_rate, training=self.training)
        output, (hn, cn) = self.lstm(input)
        output = torch.cat((hn[0],hn[1]), dim=1)
        output = F.dropout(output, p=self.dropout_rate, training=self.training)
        output = self.out(output)
        return F.log_softmax(output, dim=1)

## Training

In [ ]:
#hyperparameters
num_epochs = 12
batch_size = 64
hunits = 256
word_embed_dim = 200
pos_embed_dim= 50
dropout_rate = 0.5
lr = 0.003

#model
model = NRE2(max_len, len(vocab)+2, hunits, word_embed_dim, pos_embed_dim, dropout_rate, y_train.shape[1])
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

#evaluation function
def evaluate2(data, targets, return_out=False):
    X = index_words(data).to(device)
    pos1, pos2 = word_distances(data)
    pos1, pos2 = pos1.to(device), pos2.to(device)
    with torch.no_grad():
        model.eval()
        out = model(X, pos1, pos2).cpu()
        loss = F.nll_loss(input=out, target=targets.argmax(dim=1))
        f1 = f1_score(targets.argmax(dim=1), out.argmax(dim=1), average='macro')
    if return_out:
        return loss, f1, out
    else:
        return loss, f1

#training loop
train_losses = []
train_f1s = []
val_losses = []
val_f1s = []
for epoch in range(num_epochs):
    #training
    loss_sum = 0
    train_preds = []
    train_targets = []
    model.train()
    for i in range(0, train.shape[0], batch_size):
        X_train_batch = index_words(train.iloc[i:i+batch_size]).to(device)
        pos1_batch, pos2_batch = word_distances(train.iloc[i:i+batch_size])
        pos1_batch, pos2_batch = pos1_batch.to(device), pos2_batch.to(device)
        y_train_batch = y_train[i:i+batch_size].to(device)
        train_targets.append(y_train_batch.cpu())
        optimizer.zero_grad()
        out = model(X_train_batch, pos1_batch, pos2_batch)
        train_preds.append(out.detach().cpu())
        loss = F.nll_loss(input=out, target=y_train_batch.argmax(dim=1))
        loss.backward()
        optimizer.step()
        loss_sum += loss.detach().item()
    train_targets = torch.cat(train_targets)
    train_preds = torch.cat(train_preds)
    f1_train = f1_score(train_targets.argmax(dim=1), train_preds.argmax(dim=1), average='macro')
    train_f1s.append(f1_train)
    train_losses.append(loss_sum/batch_size)
    #validation
    loss_val, f1_val = evaluate2(val, y_val)
    val_losses.append(loss_val)
    val_f1s.append(f1_val)
    #epoch end
    print(f'********Epoch {epoch+1}********')
    print(f'Train loss: {loss_sum/batch_size:.2f}, Train F1: {f1_train:.2f};    Val loss {loss_val:.2f}, Val F1 {f1_val:.2f}\n')

#training end
plt.figure()
plt.plot(train_losses)
plt.plot(val_losses)
plt.legend(['train', 'val'])
plt.xlabel('epochs')
plt.ylabel('loss')
plt.show()

plt.figure()
plt.plot(train_f1s)
plt.plot(val_f1s)
plt.legend(['train', 'val'])
plt.xlabel('epochs')
plt.ylabel('F1')
plt.show()

## Testing

In [ ]:
loss_test, f1_test, out = evaluate2(test, y_test, return_out=True)
print(f'Test loss: {loss_test:.2f}, Test F1: {f1_test:.2f}\n')
ConfusionMatrixDisplay.from_predictions(
    y_test.argmax(dim=1),
    out.argmax(dim=1),
    display_labels=ohe.categories_[0],
    xticks_rotation='vertical',
    cmap='Blues'
);

## Comparing the Two Model

If we compare the two models based on the learning curves at first, we see that the first one presentes a larger gap between the training and validation losses with the former being higher. This model has clearly overfit the training data. Although the second model also presents a validation gap albeit smaller, there is an opportunity to stop at epoch 8 to eliminate this while the firts model starts to overfit much earlier.

In terms of test performance, there is a clear advantage when introducing the relative word positions in terms of F1 score.

Looking at the confusion matrices, the predictions for the second model are more concentrated  along the diagonal which indicates more correct predictions.

One thing that was problematic for both both models though was the "Other" class. Both models often mistook relationships of type "Other" for actual relationships since the prediction are scattered and not concentrated on the diagonal element. This could be due to samples of type "Other" slightly invoking types of relationships that are present in the dataset.